In [1]:
import cv2
import numpy

# Create Training Data

In [1]:
import cv2
import os

# Ensure the output directory exists
output_dir = './face/user_1/'
os.makedirs(output_dir, exist_ok=True)

# Load HAAR face classifier
face_classifier = cv2.CascadeClassifier('model/haarcascade-frontalface-default.xml')

# Load function
def face_extractor(img):
    # Function detects faces and returns cropped face
    # If no face detected it returns None

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)

    if len(faces) == 0:
        return None

    # Crop the first face found
    for (x, y, w, h) in faces:
        cropped_face = gray[y:y+h, x:x+w]
        return cropped_face

# Initialize Webcam
cap = cv2.VideoCapture(0)
count = 0

# Collect 100 samples of your face from webcam input
while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture image")
        break

    face = face_extractor(frame)
    if face is not None:
        count += 1
        face = cv2.resize(face, (200, 200))

        # Save file in specified directory with a unique name
        file_name_path = os.path.join(output_dir, f'{count}.jpg')
        cv2.imwrite(file_name_path, face)

        # Put count on images and display live count
        cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)  # Text color is white for grayscale
        cv2.imshow('Face Cropper', face)
    else:
        print("Face Not Found!!")
        pass

    if cv2.waitKey(1) == 13 or count == 100:  # Press 'Enter' to exit or collect 100 samples
        break

cap.release()
cv2.destroyAllWindows()
print("Collecting samples complete")


Face Not Found!!
Face Not Found!!
Face Not Found!!
Face Not Found!!
Face Not Found!!
Face Not Found!!
Face Not Found!!
Face Not Found!!
Face Not Found!!
Face Not Found!!
Face Not Found!!
Face Not Found!!
Face Not Found!!
Face Not Found!!
Face Not Found!!
Face Not Found!!
Face Not Found!!
Face Not Found!!
Face Not Found!!
Face Not Found!!
Face Not Found!!
Face Not Found!!
Face Not Found!!
Face Not Found!!
Face Not Found!!
Face Not Found!!
Face Not Found!!
Face Not Found!!
Face Not Found!!
Face Not Found!!
Face Not Found!!
Face Not Found!!
Face Not Found!!
Face Not Found!!
Face Not Found!!
Face Not Found!!
Face Not Found!!
Face Not Found!!
Face Not Found!!
Face Not Found!!
Face Not Found!!
Face Not Found!!
Face Not Found!!
Face Not Found!!
Face Not Found!!
Face Not Found!!
Face Not Found!!
Face Not Found!!
Face Not Found!!
Face Not Found!!
Face Not Found!!
Face Not Found!!
Face Not Found!!
Face Not Found!!
Face Not Found!!
Face Not Found!!
Face Not Found!!
Face Not Found!!
Face Not Found

# Train Model

In [2]:
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

# Get training data we previously made
data_path = './face/user_1/'
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]

# Create arrays for training data and labels
Training_Data, Labels = [], []

# Open Training data in datasets
# Create a numpy array for training data
for i, file in enumerate(onlyfiles):
    image_path = join(data_path, file)
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if images is None:
        continue
    Training_Data.append(np.asarray(images, dtype=np.uint8))
    Labels.append(i)

# Creating numpy array for both training data and label
Labels = np.asarray(Labels, dtype=np.int32)

# Ensure that we have some training data
if len(Training_Data) == 0:
    print("No training data found. Please ensure there are images in the specified directory.")
    exit()

# Initialize face recognizer
model = cv2.face.LBPHFaceRecognizer_create()

# Train the model
model.train(np.asarray(Training_Data), np.asarray(Labels))
print("Model trained successfully")


Model trained successfully


# Run Face Recog

In [3]:
import cv2
import numpy as np

# Load the pre-trained face detection classifier
face_classifier = cv2.CascadeClassifier('model/haarcascade-frontalface-default.xml')

def face_extractor(img):
    # Function detects faces and returns the cropped face
    # If no face is detected, it returns None
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is not None:
        # Crop the first face found
        for (x, y, w, h) in faces:
            cropped_face = img[y:y+h, x:x+w]
            return cropped_face
    return None

def face_detector(img, size=0.5):
    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is not None:
        for (x, y, w, h) in faces:
            cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 255), 2)
            roi = img[y:y+h, x:x+w]
            roi = cv2.resize(roi, (200, 200))
            return img, roi  # Return the image and roi immediately after finding the first face
    return img, []  # Return the original image and an empty list if no face is found
    
# Open the webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    image, face = face_detector(frame)

    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        # Pass the face to the prediction model
        # "results" should contain a tuple with the label and confidence value
        results = model.predict(face)  # Assuming 'model' is defined and loaded

        if results[1] < 500:
            confidence = int(100 * (1 - (results[1]) / 400))
            display_string = str(confidence) + '% Confidence it is true'

        cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 120, 150), 2)

        if confidence > 75:
            cv2.putText(image, "Unlocked", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
            cv2.imshow('Face Recognition', image)
        else:
            cv2.putText(image, "Locked", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)
            cv2.imshow('Face Recognition', image)

    except Exception as e:
        # Handle any exceptions that occur during prediction
        print(f"An error occurred: {e}")
        cv2.putText(image, "No Face Found", (220, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)
        cv2.putText(image, "Locked", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)
        cv2.imshow('Face Recognition', image)

    # Press 'Enter' to exit the loop
    if cv2.waitKey(1) == 13:
        break

# Release the webcam and close all windows
cap.release()
cv2.destroyAllWindows()

An error occurred: OpenCV(4.10.0) :-1: error: (-5:Bad argument) in function 'cvtColor'
> Overload resolution failed:
>  - src is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src'

An error occurred: OpenCV(4.10.0) :-1: error: (-5:Bad argument) in function 'cvtColor'
> Overload resolution failed:
>  - src is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src'

An error occurred: OpenCV(4.10.0) :-1: error: (-5:Bad argument) in function 'cvtColor'
> Overload resolution failed:
>  - src is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src'

An error occurred: OpenCV(4.10.0) :-1: error: (-5:Bad argument) in function 'cvtColor'
> Overload resolution failed:
>  - src is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src'

An error occurred: OpenCV(4.10.0) :-1: error: (-5:Bad argument) in function 'cvtColor'
> Overload resolution failed:
>  - src is not a numpy array, 

KeyboardInterrupt: 